In [1]:
import pandas as pd
import requests
import json
import numpy as np
import os
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
from pykrx import stock
import dart_fss as dart
from selenium import webdriver
import re
from io import StringIO
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
pd.options.display.float_format = '{:.6f}'.format

In [2]:
cdf = pd.read_csv('data/company_name_and_code.csv', index_col = 0)
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,95570
1,125080,AK홀딩스,6840
2,219097,BGF,27410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,5870
727,362238,휴비스,79980
728,156488,휴스틸,5010
729,103176,흥국화재,540


In [3]:
cdf['stock_code'] = cdf['stock_code'].astype(str).str.zfill(6)

In [4]:
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,095570
1,125080,AK홀딩스,006840
2,219097,BGF,027410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,005870
727,362238,휴비스,079980
728,156488,휴스틸,005010
729,103176,흥국화재,000540


In [5]:
stock_code = list(cdf['stock_code'])

In [6]:
# driver = webdriver.Chrome()
# driver.get("https://dart.fss.or.kr/")
# 

In [7]:
driver = webdriver.Chrome("")
url = 'https://dart.fss.or.kr/dsab001/main.do'
driver.get(url)

In [8]:
wait = WebDriverWait(driver, 10)

In [9]:
# ===============================
# 필수 import
# ===============================
import re
import time
import os
import pandas as pd
from io import StringIO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

# ===============================
# 테이블에서 광고비 추출
# ===============================
def extract_from_table_advertising(driver):
    try:
        driver.switch_to.default_content()
        WebDriverWait(driver, 5).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrm"))
        )

        time.sleep(0.5)
        page_text = driver.find_element(By.TAG_NAME, "body").text
        ad_keywords = ['광고선전비', '광고비', '광고료']

        if not any(k in page_text for k in ad_keywords):
            return None

        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.TAG_NAME, "table"))
            )
        except TimeoutException:
            return None

        tables = pd.read_html(StringIO(driver.page_source))

        unit = 1
        t = page_text.replace(" ", "")
        if "단위:천원" in t or "(천원)" in t:
            unit = 1000
        elif "단위:백만원" in t or "(백만원)" in t:
            unit = 1_000_000
        elif "단위:억원" in t or "(억원)" in t:
            unit = 100_000_000

        pattern = r'광고선전비|광고비|광고료'

        for df in tables:
            if df.empty or len(df.columns) < 2:
                continue

            col0 = df.iloc[:, 0].astype(str).str.replace(r'\s+', '', regex=True)
            mask = col0.str.contains(pattern, regex=True, na=False)

            if mask.any():
                row = df[mask].iloc[0]
                for i in range(1, min(len(df.columns), 4)):
                    raw_val = str(row.iloc[i])
                    num = re.sub(r'[^\d.-]', '', raw_val)
                    if num and num not in ['.', '-']:
                        try:
                            val = float(num) * unit
                            if val > 0:
                                return val
                        except:
                            continue
        return None
    except:
        return None

# ===============================
# 주석 탭 순회 (Stale 방지)
# ===============================
def extract_advertising_expense(driver):
    try:
        tab_index = 0
        while True:
            driver.switch_to.default_content()
            note_tabs = driver.find_elements(
                By.XPATH,
                "//*[@id='left-panel']//*[contains(text(), '주석')]"
            )

            if tab_index >= len(note_tabs):
                break

            target_tab = note_tabs[tab_index]
            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", target_tab)
                time.sleep(0.3)
                driver.execute_script("arguments[0].click();", target_tab)
                time.sleep(1.5)

                val = extract_from_table_advertising(driver)
                if val is not None and val > 0:
                    return val
                
                tab_index += 1
            except StaleElementReferenceException:
                continue
            except:
                tab_index += 1
                continue
        return 0.0
    except:
        return 0.0

# ===============================
# 메인 실행 함수 (연도별 CSV 저장 및 종료)
# ===============================
def run_and_shutdown_csv_final(cdf, start_date=2019, end_date=2024):
    print(f"🚀 [시작] 총 {len(cdf)}개 기업 분석을 시작합니다.")
    
    yearly_dfs = {y: cdf.copy() for y in range(start_date, end_date + 1)}
    for y in yearly_dfs:
        yearly_dfs[y]['advertising_expenses'] = 0.0

    wait = WebDriverWait(driver, 10)

    try:
        for n, code in enumerate(cdf['stock_code']):
            print(f"\n🏢 [{n+1}/{len(cdf)}] 기업코드: {code}")
            for year in range(start_date, end_date + 1):
                try:
                    driver.switch_to.window(driver.window_handles[0])
                    inp = wait.until(EC.element_to_be_clickable((By.ID, 'textCrpNm')))
                    inp.clear()
                    inp.send_keys(code)

                    driver.find_element(By.ID, 'startDate').clear()
                    driver.find_element(By.ID, 'startDate').send_keys(f'{year+1}0101')
                    driver.find_element(By.ID, 'endDate').clear()
                    driver.find_element(By.ID, 'endDate').send_keys(f'{year+1}1231')

                    try:
                        driver.find_element(By.CSS_SELECTOR, '#li_01 > label > span').click()
                    except: pass

                    cb = wait.until(EC.presence_of_element_located((By.ID, 'publicTypeDetail_A001')))
                    if not cb.is_selected():
                        driver.execute_script("arguments[0].click();", cb)

                    driver.find_element(By.XPATH, "//a[@title='검색']").click()
                    time.sleep(1.5)

                    try:
                        driver.find_element(By.XPATH, '//*[@id="listContents"]//td[3]/a').click()
                    except:
                        print(f"    📅 {year}년: [미공시]")
                        continue

                    time.sleep(2.5)
                    driver.switch_to.window(driver.window_handles[-1])

                    ad = extract_advertising_expense(driver)
                    yearly_dfs[year].at[n, 'advertising_expenses'] = ad
                    print(f"    📅 {year}년: 성공 (광고:{ad:,.0f})")

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                except Exception:
                    if len(driver.window_handles) > 1:
                        driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            # 10개마다 중간 백업
            if (n + 1) % 10 == 0:
                for y, df in yearly_dfs.items():
                    df.to_csv(f'data/advertising_data_{y}_temp.csv', index=False, encoding='utf-8-sig')

    finally:
        # 최종 결과 저장 (연도별 CSV 저장)
        print("\n💾 [최종 저장] 작업을 완료하고 CSV 파일들을 저장합니다.")
        for y, df in yearly_dfs.items():
            # 요청하신 경로 형식 적용
            file_path = f'data/advertising_data_{y}.csv'
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
            print(f"✅ 저장 완료: {file_path}")
        
        driver.quit()
        
        # 시스템 종료 (Windows)
        print("🛑 60초 후 컴퓨터가 종료됩니다. 취소하려면 'shutdown -a'를 입력하세요.")
        os.system("shutdown /s /t 60") 

# ===============================
# 실행부
# ===============================
driver = webdriver.Chrome()
driver.get("https://dart.fss.or.kr/dsab001/main.do")

run_and_shutdown_csv_final(cdf)

🚀 [시작] 총 731개 기업 분석을 시작합니다.

🏢 [1/731] 기업코드: 095570
    📅 2019년: 성공 (광고:4,916,319,000)
    📅 2020년: 성공 (광고:3,323,899,000)
    📅 2021년: 성공 (광고:2,076,794,000)
    📅 2022년: 성공 (광고:1,737,900,000)
    📅 2023년: 성공 (광고:2,288,799,000)
    📅 2024년: 성공 (광고:744,191,000)

🏢 [2/731] 기업코드: 006840
    📅 2019년: 성공 (광고:63,748,087,000)
    📅 2020년: 성공 (광고:65,432,074,000)
    📅 2021년: 성공 (광고:70,572,823,000)
    📅 2022년: 성공 (광고:76,460,769,000)
    📅 2023년: 성공 (광고:91,555,618,000)
    📅 2024년: 성공 (광고:105,297,623,000)

🏢 [3/731] 기업코드: 027410
    📅 2019년: [미공시]
    📅 2020년: 성공 (광고:4,578,000,000)
    📅 2021년: 성공 (광고:5,904,000,000)
    📅 2022년: 성공 (광고:4,259,000,000)
    📅 2023년: 성공 (광고:2,930,000,000)
    📅 2024년: 성공 (광고:2,596,000,000)

🏢 [4/731] 기업코드: 282330
    📅 2019년: 성공 (광고:2,097,000,000)
    📅 2020년: 성공 (광고:3,536,000,000)
    📅 2021년: 성공 (광고:3,879,000,000)
    📅 2022년: 성공 (광고:4,743,000,000)
    📅 2023년: 성공 (광고:4,477,000,000)
    📅 2024년: 성공 (광고:5,128,000,000)

🏢 [5/731] 기업코드: 138930
    📅 2019년: 성공 (광고:4,2

    📅 2019년: [미공시]
    📅 2020년: 성공 (광고:0)
    📅 2021년: 성공 (광고:0)
    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [39/731] 기업코드: 010620
    📅 2019년: 성공 (광고:2,076,000,000)
    📅 2020년: 성공 (광고:1,028,000,000)
    📅 2021년: 성공 (광고:1,070,000,000)
    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [40/731] 기업코드: 042670
    📅 2019년: 성공 (광고:79,751,000,000)
    📅 2020년: 성공 (광고:67,636,000,000)
    📅 2021년: 성공 (광고:38,812,000,000)
    📅 2022년: 성공 (광고:14,233,000,000)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [41/731] 기업코드: 267260
    📅 2019년: 성공 (광고:2,557,000,000)
    📅 2020년: 성공 (광고:1,468,000,000)
    📅 2021년: 성공 (광고:2,218,000,000)
    📅 2022년: 성공 (광고:3,883,000,000)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [42/731] 기업코드: 097230
    📅 2019년: 성공 (광고:429,000,000)
    📅 2020년: 성공 (광고:477,000,000)
    📅 2021년: 성공 (광고:802,000,000)
    📅 2022년: 성공 (광고:2,154,000,000)
    📅 2023년: 성공 (광고:2,052,000,000)
    📅 2024년: 성공 (광고:2,115,000,000)

🏢 [43/731] 

    📅 2021년: 성공 (광고:40,500,000)
    📅 2022년: 성공 (광고:24,500,000)
    📅 2023년: 성공 (광고:27,800,000)
    📅 2024년: 성공 (광고:19,800,000)

🏢 [88/731] 기업코드: 035420
    📅 2019년: 성공 (광고:884,405,680,000)
    📅 2020년: 성공 (광고:762,604,197,000)
    📅 2021년: 성공 (광고:1,071,730,155,000)
    📅 2022년: 성공 (광고:1,300,591,276,000)
    📅 2023년: 성공 (광고:1,479,801,562,000)
    📅 2024년: 성공 (광고:1,599,360,016,000)

🏢 [89/731] 기업코드: 181710
    📅 2019년: 성공 (광고:62,408,874,000)
    📅 2020년: 성공 (광고:77,730,506,000)
    📅 2021년: 성공 (광고:88,249,972,000)
    📅 2022년: 성공 (광고:126,756,537,000)
    📅 2023년: 성공 (광고:78,322,604,000)
    📅 2024년: 성공 (광고:1,869,000)

🏢 [90/731] 기업코드: 005940
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광고:0)
    📅 2021년: 성공 (광고:0)
    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [91/731] 기업코드: 034310
    📅 2019년: 성공 (광고:28,117,075,000)
    📅 2020년: 성공 (광고:32,670,498,000)
    📅 2021년: 성공 (광고:38,616,298,000)
    📅 2022년: 성공 (광고:41,593,988,000)
    📅 2023년: 성공 (광고:37,882,706,000)
    📅 2024년:

    📅 2024년: 성공 (광고:0)

🏢 [124/731] 기업코드: 011810
    📅 2019년: 성공 (광고:159,000,000)
    📅 2020년: 성공 (광고:41,000,000)
    📅 2021년: 성공 (광고:61,000,000)
    📅 2022년: 성공 (광고:87,000,000)
    📅 2023년: 성공 (광고:226,000,000)
    📅 2024년: 성공 (광고:163,000,000)

🏢 [125/731] 기업코드: 077970
    📅 2019년: 성공 (광고:328,260,000)
    📅 2020년: 성공 (광고:225,294,000)
    📅 2021년: 성공 (광고:530,861,000)
    📅 2022년: 성공 (광고:548,862,000)
    📅 2023년: 성공 (광고:714,689,000)
    📅 2024년: 성공 (광고:0)

🏢 [126/731] 기업코드: 002820
    📅 2019년: 성공 (광고:297,557,000)
    📅 2020년: 성공 (광고:389,570,000)
    📅 2021년: 성공 (광고:353,767,000)
    📅 2022년: 성공 (광고:312,080,000)
    📅 2023년: 성공 (광고:541,348,000)
    📅 2024년: 성공 (광고:86,145,000)

🏢 [127/731] 기업코드: 002710
    📅 2019년: 성공 (광고:27,354,139,000)
    📅 2020년: 성공 (광고:2,309,090,000)
    📅 2021년: 성공 (광고:27,185,090,000)
    📅 2022년: 성공 (광고:40,309,090,000)
    📅 2023년: 성공 (광고:103,109,090,000)
    📅 2024년: 성공 (광고:69,538,090,000)

🏢 [128/731] 기업코드: 007980
    📅 2019년: 성공 (광고:891,794,000)
    📅 2020년: 성공 (광

    📅 2024년: 성공 (광고:188,928,000)

🏢 [162/731] 기업코드: 001570
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광고:0)
    📅 2021년: 성공 (광고:0)
    📅 2022년: [미공시]
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [163/731] 기업코드: 002990
    📅 2019년: 성공 (광고:1,363,000,000)
    📅 2020년: 성공 (광고:1,194,000,000)
    📅 2021년: 성공 (광고:1,105,000,000)
    📅 2022년: 성공 (광고:976,000,000)
    📅 2023년: 성공 (광고:1,036,000,000)
    📅 2024년: 성공 (광고:3,170,000,000)

🏢 [164/731] 기업코드: 011780
    📅 2019년: 성공 (광고:2,301,000,000)
    📅 2020년: 성공 (광고:7,575,000,000)
    📅 2021년: 성공 (광고:10,727,000,000)
    📅 2022년: 성공 (광고:12,790,000,000)
    📅 2023년: 성공 (광고:11,464,000,000)
    📅 2024년: 성공 (광고:11,392,000,000)

🏢 [165/731] 기업코드: 214330
    📅 2019년: 성공 (광고:38,981,000)
    📅 2020년: 성공 (광고:47,836,000)
    📅 2021년: 성공 (광고:20,894,000)
    📅 2022년: 성공 (광고:44,700,000)
    📅 2023년: 성공 (광고:35,671,000)
    📅 2024년: 성공 (광고:80,448,000)

🏢 [166/731] 기업코드: 001210
    📅 2019년: 성공 (광고:747,270,000)
    📅 2020년: 성공 (광고:300,609,000)
    📅 2021년: 성공 (광고:815

...

    📅 2023년: 성공 (광고:307,481,000,000)
    📅 2024년: 성공 (광고:341,302,000,000)

🏢 [558/731] 기업코드: 007810
    📅 2019년: 성공 (광고:9,823,000)
    📅 2020년: 성공 (광고:10,106,000)
    📅 2021년: 성공 (광고:42,035,000)
    📅 2022년: 성공 (광고:83,323,000)
    📅 2023년: 성공 (광고:20,877,000)
    📅 2024년: 성공 (광고:34,286,578,000)

🏢 [559/731] 기업코드: 003690
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광고:0)
    📅 2021년: 성공 (광고:0)
    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [560/731] 기업코드: 192820
    📅 2019년: 성공 (광고:2,797,280,807)
    📅 2020년: 성공 (광고:3,339,893,807)
    📅 2021년: 성공 (광고:1,388,794,222)
    📅 2022년: 성공 (광고:2,046,764,291)
    📅 2023년: 성공 (광고:3,247,058,067)
    📅 2024년: 성공 (광고:0)

🏢 [561/731] 기업코드: 044820
    📅 2019년: 성공 (광고:968,484,652,000)
    📅 2020년: 성공 (광고:996,806,110)
    📅 2021년: 성공 (광고:5,900,637,650)
    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:2,232,264,876)
    📅 2024년: 성공 (광고:1,543,054,963)

🏢 [562/731] 기업코드: 005070
    📅 2019년: [미공시]
    📅 2020년: 성공 (광고:19,737,000)
    📅 2021년: 성공 

    📅 2019년: 성공 (광고:145,572,000)
    📅 2020년: 성공 (광고:12,495,000)
    📅 2021년: 성공 (광고:80,546,000)
    📅 2022년: 성공 (광고:39,936,000)
    📅 2023년: 성공 (광고:452,960,000)
    📅 2024년: 성공 (광고:110,771,000)

🏢 [600/731] 기업코드: 016800
    📅 2019년: 성공 (광고:3,814,609,686)
    📅 2020년: 성공 (광고:1,865,572,068)
    📅 2021년: 성공 (광고:2,697,245,166)
    📅 2022년: 성공 (광고:1,607,413,325,000)
    📅 2023년: 성공 (광고:2,396,544,182)
    📅 2024년: 성공 (광고:0)

🏢 [601/731] 기업코드: 001020
    📅 2019년: 성공 (광고:2,354,230,000)
    📅 2020년: 성공 (광고:1,576,074,000)
    📅 2021년: 성공 (광고:2,785,037,000)
    📅 2022년: 성공 (광고:821,070,000)
    📅 2023년: 성공 (광고:794,453,000)
    📅 2024년: 성공 (광고:622,314,000)

🏢 [602/731] 기업코드: 090080
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광고:0)
    📅 2021년: [미공시]

🏢 [603/731] 기업코드: 010770

🏢 [604/731] 기업코드: 022100

🏢 [605/731] 기업코드: 058430

🏢 [606/731] 기업코드: 047050
    📅 2022년: 성공 (광고:1,733,000)
    📅 2023년: 성공 (광고:4,331,000)
    📅 2024년: 성공 (광고:5,092,000)

🏢 [607/731] 기업코드: 003670
    📅 2019년: 성공 (광고:1,247,511,000

    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [640/731] 기업코드: 007280
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광고:0)
    📅 2021년: 성공 (광고:0)
    📅 2022년: [미공시]
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [641/731] 기업코드: 047810
    📅 2019년: 성공 (광고:4,684,352,000)
    📅 2020년: 성공 (광고:1,751,231,000)
    📅 2021년: 성공 (광고:6,240,235,000)
    📅 2022년: 성공 (광고:7,671,574,000)
    📅 2023년: 성공 (광고:13,770,237,000)
    📅 2024년: 성공 (광고:7,904,384,000)

🏢 [642/731] 기업코드: 123690
    📅 2019년: 성공 (광고:8,548,739,000)
    📅 2020년: 성공 (광고:4,629,628,000)
    📅 2021년: 성공 (광고:1,823,373,000)
    📅 2022년: 성공 (광고:1,480,408,000)
    📅 2023년: 성공 (광고:3,365,482,000)
    📅 2024년: 성공 (광고:7,153,414,000)

🏢 [643/731] 기업코드: 003350
    📅 2019년: 성공 (광고:39,748,000)
    📅 2020년: 성공 (광고:36,802,000)
    📅 2021년: 성공 (광고:38,364,000)
    📅 2022년: 성공 (광고:143,896,000)
    📅 2023년: 성공 (광고:37,082,000)
    📅 2024년: 성공 (광고:29,764,000)

🏢 [644/731] 기업코드: 011500
    📅 2019년: 성공 (광고:53,880,745)
    📅 2020년: 성공 (광고:2

    📅 2019년: [미공시]
    📅 2020년: 성공 (광고:1,704,380,000)
    📅 2021년: 성공 (광고:820,527,000)
    📅 2022년: 성공 (광고:459,488,000)
    📅 2023년: 성공 (광고:395,742,000)
    📅 2024년: 성공 (광고:280,327,000)

🏢 [677/731] 기업코드: 005110
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광고:0)
    📅 2021년: 성공 (광고:0)
    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:18,803,000)

🏢 [678/731] 기업코드: 009460
    📅 2019년: 성공 (광고:23,740,000)
    📅 2020년: 성공 (광고:23,292,000)
    📅 2021년: 성공 (광고:35,875,000)
    📅 2022년: 성공 (광고:46,295,000)
    📅 2023년: 성공 (광고:41,104,000)
    📅 2024년: 성공 (광고:53,626,000)

🏢 [679/731] 기업코드: 000880
    📅 2019년: 성공 (광고:96,586,000,000)
    📅 2020년: 성공 (광고:101,832,000,000)
    📅 2021년: 성공 (광고:78,117,000)
    📅 2022년: [미공시]
    📅 2023년: [미공시]
    📅 2024년: 성공 (광고:0)

🏢 [680/731] 기업코드: 088350
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광고:0)
    📅 2021년: 성공 (광고:0)
    📅 2022년: 성공 (광고:0)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [681/731] 기업코드: 000370
    📅 2019년: 성공 (광고:0)
    📅 2020년: 성공 (광

    📅 2019년: 성공 (광고:46,802,000)
    📅 2020년: 성공 (광고:43,626,000)
    📅 2021년: 성공 (광고:52,000,000)
    📅 2022년: 성공 (광고:82,449,000)
    📅 2023년: 성공 (광고:110,372,000)
    📅 2024년: 성공 (광고:97,327,000)

🏢 [716/731] 기업코드: 133820
    📅 2019년: 성공 (광고:14,660,000)
    📅 2020년: 성공 (광고:15,505,000)
    📅 2021년: 성공 (광고:16,414,000)
    📅 2022년: 성공 (광고:46,415,000)
    📅 2023년: 성공 (광고:0)
    📅 2024년: 성공 (광고:0)

🏢 [717/731] 기업코드: 010660
    📅 2019년: 성공 (광고:365,556,000)
    📅 2020년: 성공 (광고:74,804,000)
    📅 2021년: 성공 (광고:102,666,000)
    📅 2022년: 성공 (광고:596,403,000)
    📅 2023년: 성공 (광고:339,573,000)
    📅 2024년: 성공 (광고:312,179,000)

🏢 [718/731] 기업코드: 000850
    📅 2019년: 성공 (광고:1,734,533,000)
    📅 2020년: 성공 (광고:669,611,000)
    📅 2021년: 성공 (광고:718,067,000)
    📅 2022년: 성공 (광고:3,063,665,000)
    📅 2023년: 성공 (광고:2,249,810,000)
    📅 2024년: 성공 (광고:3,524,413,000)

🏢 [719/731] 기업코드: 016580
    📅 2019년: 성공 (광고:4,929,540,000)
    📅 2020년: 성공 (광고:7,068,525,000)
    📅 2021년: 성공 (광고:9,519,140,000)
    📅 2022년: 성공 (광고:1